# Grundlagen der KI - Gruppenprojekte


Für die Projekte stellen wir eine MongoDB Datenbank, die wir aktiv in der Forschung nutzen, zur Verfügung. Die Datenbank beinhaltet Informationen über Versionskontrollsysteme, Issuetracker, und Mailinglisten von Open Source Projekten. [Die Dokumentation der Datenbank finden Sie Online](https://smartshark2.informatik.uni-goettingen.de/documentation/).

## Die Aufgabe

Ihre Aufgabe ist es, einen automatisierten Ansatz zu entwickeln, welcher vorhersagt, ob ein Issue innerhalb von 6 Monaten behoben wird oder nicht. Darüber hinaus, sollte das Modell (oder auch ein zweites Modell) in der Lage sein, die erwartete Zeit für die Behebung und damit Schließung des Issues vorherzusagen.
Sie können dazu beliebige Informationen aus der Datenbank nutzen, wie z.B. Commits, andere Issues, Mails und Änderungen in den Dateien.
Sie müssen diese Problemstellung als Analyseproblem fassen um dann Modelle zur Lösung aufstellen zu können. Hierzu müssen Sie geeignete Analysemethoden auswählen und anschließend die Qualität der Ergebnisse evaluieren. 

## Präsentationen

**Alle Gruppen müssen ihre Ergebnisse in der letzten Vorlesung am 09.02. um 16:15 Uhr vorstellen.** Hierzu muss jede Gruppe eine kurze Präsentation halten. In dieser Präsentation sollten Sie kurz beschreiben wie Sie vorgegangen sind, wie Sie Features erstellt haben, welche Modelle Sie benutzt haben und was ihre wichtigsten Ergebnisse sind. Die Zeit für die Präsentation beträgt 9 (1) Minuten.

Im Anschluss an die Präsentation stimmen die Anwesenden über das beste Projekt ab. Jede Gruppe stimmt für das beste Projekt (3 Punkte), das zweitbeste (2 Punkte) und das drittbeste (1 Punkt). Das Projekt mit den meisten Punkten gewinnt einen Preis.

Die Präsentation wird in Präsenz stattfinden, wenn die aktuellen Regeln es zulassen (Prof. Herbold wird dabei per Video zugeschaltet sein; Raum wird noch bekannt gegeben). Da wir nur begrenzt Zeit haben, müssen alle Präsentationen vorab geschickt werden. Wir kombinieren diese dann in einer PDF um einen einfachen Wechsel zwischen den Präsentation der Gruppen zu erreichen. Die Mitglieder der Gruppe werden dann in der Reihenfolge des Gruppennames gebeten ihre Ergebnisse vorzustellen, d.h. wir beginnen mit Gruppe 1 und enden mit Gruppe 6.


## Abgabe der Präsentationen

Damit die Präsentation reibungslos ablaufen, müssen Sie ihre Präsentation im Vorfeld einsenden. **Schicken Sie Ihre Präsentationen bitte bis spätestens zum 08.02.23, 16 Uhr an Clemens Dierks via Email (clemens.dierks@tu-clausthal.de)**. Die Präsentation muss als PDF vorliegen.


## Mindestkriterien

Die folgenden Kriterien muss ein Projekt mindestens erfüllen, damit die Gruppenmitglieder an der Prüfung teilnehmen können.
- Es muss ein Modell für die automatisierte Vorhersage, ob ein Issue innerhalb von 6 Monaten gelöst wird, vorhanden sein.
- Es muss ein Modell für die automatisierte Vorhersage der estimated-time-to-fix von Issues vorhanden sein.
- Die Qualität des Modells muss bewertet werden.
- Sie müssen eine Empfehlung geben, ob und ggf. unter welchen Umständen, das Modell eingesetzt werden kann basierend auf der Evaluation der Qualität des Modells.
- Aufgrund Ihrer Empfehlung können Sie noch Verbesserungsmöglichkeiten vorstellen.
- Die Ergebnisse müssen in der Vorlesung am 09.02.23 vorgestellt werden.

## Datenbankzugriff

Unterhalb finden Sie ein Beispiel, wie Sie auf die Datenbank zugreifen. Die Datenbank befindet sich hinter einer Firewall und kann nur aus dem Netz der TU Clausthal erreicht werden. Wenn Sie Probleme haben die Datenbank zu erreichen, können Sie von jedem beliebigen Netzwerk aus in das Netz der TU Clausthal gelangen, in dem Sie sich mit dem [VPN](https://doku.tu-clausthal.de/doku.php?id=netzwerk_und_internet:vpn:start) verbinden.

**WARNUNG:
Wir benutzen die Datenbank aktiv in der Forschung. Es kann daher sein, das teilweise eine sehr hohe Last auf der Datenbank ist. Die Datenbank ist sehr groß und beinhaltet derzeit 4.8 Terabyte an Daten. Sie sollten daher nicht einfach starten Anfragen zu schreiben, sondern sich genau überlegen auf welche Informationen Sie zugreifen wollen. Wenn Sie zum Beispiel eine Anfrage stellen, in der Sie die komplette ```commit``` Collection herunterladen möchten, müssen Sie hierfür 158 Gigabyte an Daten runterladen. **

Sie können die Bibliothek [pycoSHARK](https://github.com/smartshark/pycoshark) zum Zugriff auf die Datenbank nutzen. Der pycoSHARK stellt einen ORM-Layer basierend auf der Bibliothek ```mongoengine``` zur Verfügung. Hierdurch können Sie die Datenbanktabellen wie Objekte behandeln. Alternativ können Sie die native MongoDB API der ```pymongo``` Bibliothek benutzen. 

Der Quelltext unten beinhaltet Ihre Zugangsdaten in zeigt wie man mit dem pycoSHARK auf die Datenbank zugreift.

In [6]:
# Installation des pycoSHARKs
import sys
# !{sys.executable} -m pip install pycoshark

In [7]:
from mongoengine import connect, disconnect
from pycoshark.mongomodels import File, FileAction, Commit, Project, VCSSystem, IssueSystem,Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
import pandas as pd

# Database credentials
user = 'grundlagenki2021'
password = '38BTGx71d'
host = '134.76.81.151'
port = '27017'
authentication_db = 'smartshark'
database = "smartshark"
ssl_enabled = None

# Establish connection
uri = create_mongodb_uri_string(user, password, host, port, authentication_db, ssl_enabled)
connect(database, host=uri)

# Fetch project id and issue tracking system system id for the 'commons-math' project
# The only() decides the data that is actually retrieved from the MongoDB. 
# get() only works, when you query results in exactly one document, otherwise you have to use a loop to access the data
# an example with a loop is at the bottom of this example
project = Project.objects(name='commons-math').only('id').get()
issue_system = IssueSystem.objects(project_id=project.id).only('id','url').get()
print('url of issue system of the project: %s' % issue_system.url)

url of issue system of the project: https://issues.apache.org/jira/rest/api/2/search?jql=project=MATH


Copied from https://github.com/smartshark/usage-examples/blob/main/Example-Notebook.ipynb:

In [5]:
# We first need the the project from the database
project = Project.objects(name='giraph').get()

# We now select the issue tracking system of the project
# Please note that some projects have multiple issue trackers
# In this case get() would fail and you would need to loop over them
issue_tracker = IssueSystem.objects(project_id=project.id).get()

print('Issue Tracker:', issue_tracker.url)

# we can now work with the issues
num_issues = Issue.objects(issue_system_id=issue_tracker.id).count()

print('Number of issues:', num_issues)

count_comments = 0
count_referenced_by_commits = 0
count_bugs_dev_label = 0
count_bugs_validated = 0

for issue in Issue.objects(issue_system_id=issue_tracker.id):
    count_comments += IssueComment.objects(issue_id=issue.id).count()
    if issue.issue_type is not None and issue.issue_type.lower()=='bug':
        count_bugs_dev_label += 1
    if issue.issue_type_verified is not None and issue.issue_type_verified.lower()=='bug':
        count_bugs_validated += 1
    if Commit.objects(linked_issue_ids=issue.id).count()>0:
        count_referenced_by_commits += 1
        
print('Number of comments in discussions:', count_comments)
print('Number of issues referenced by commits:', count_referenced_by_commits)
print('Number of issues labeled as bugs by developers:', count_bugs_dev_label)
print('Number of issues labeled validated as bug by researchers:', count_bugs_validated)

Issue Tracker: https://issues.apache.org/jira/rest/api/2/search?jql=project=GIRAPH
Number of issues: 1232
Number of comments in discussions: 6759
Number of issues referenced by commits: 787
Number of issues labeled as bugs by developers: 534
Number of issues labeled validated as bug by researchers: 140


In [8]:

df=pd.read_json(Issue.objects(issue_system_id=issue_tracker.id).to_json())


In [9]:
df.head

,_id,external_id,issue_system_id,affects_versions,components,labels,fix_versions,issue_links,title,desc,...,issue_type,priority,status,resolution,assignee_id,issue_type_manual,issue_type_verified,environment,original_time_estimate,parent_issue_id
0,{'$oid': '5bf58b8b423c98412cd404c9'},INFRA-3855,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[],[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5bf58b8b423c98412cd404ca'},GIRAPH-1,{'$oid': '5bf58b88423c98412cd404c8'},[0.1.0],[],[],[0.1.0],[{'issue_id': {'$oid': '5bf58b8b423c98412cd404...,Initial code import,I did the initial code import from github.,...,Task,Major,Resolved,Fixed,{'$oid': '5b3b302830a71b06bc515831'},NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5bf58b8c423c98412cd404dc'},GIRAPH-9,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[0.1.0],[],Change Yahoo License Header to Apache License ...,All source codes contains Yahoo License Header...,...,Task,Major,Resolved,Fixed,{'$oid': '5b48554530a71b06bcb992b0'},"{'sherbold': 'other', 'atrautsch': 'documentat...",other,NaN,NaN,NaN
3,{'$oid': '5bf58b8d423c98412cd404e6'},GIRAPH-7,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[],[],Remove Yahoo-specific code from pom.xml,There are remaining references to Y! infrastru...,...,Bug,Blocker,Resolved,Duplicate,{'$oid': '5b3b3d6230a71b06bc84d07f'},NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5bf58b8e423c98412cd404ea'},GIRAPH-29,{'$oid': '5bf58b88423c98412cd404c8'},[],[bsp],[],[0.1.0],[],Implement TextVertexInputFormat for text-forma...,Supporting text-format graph data would be nic...,...,New Feature,Minor,Resolved,Won't Fix,{'$oid': '5b48554530a71b06bcb992b0'},NaN,NaN,NaN,NaN,NaN


In [16]:
df1=df
df1['created_at']=pd.json_normalize(df1['created_at'])
df1['updated_at']=pd.json_normalize(df1['updated_at'])

df1['timetofix']=df1.updated_at-df1.created_at

In [20]:
df1.head()

,_id,external_id,issue_system_id,affects_versions,components,labels,fix_versions,issue_links,title,desc,...,priority,status,resolution,assignee_id,issue_type_manual,issue_type_verified,environment,original_time_estimate,parent_issue_id,timetofix
0,{'$oid': '5bf58b8b423c98412cd404c9'},INFRA-3855,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[],[],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5bf58b8b423c98412cd404ca'},GIRAPH-1,{'$oid': '5bf58b88423c98412cd404c8'},[0.1.0],[],[],[0.1.0],[{'issue_id': {'$oid': '5bf58b8b423c98412cd404...,Initial code import,I did the initial code import from github.,...,Major,Resolved,Fixed,{'$oid': '5b3b302830a71b06bc515831'},NaN,NaN,NaN,NaN,NaN,472046561.0
2,{'$oid': '5bf58b8c423c98412cd404dc'},GIRAPH-9,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[0.1.0],[],Change Yahoo License Header to Apache License ...,All source codes contains Yahoo License Header...,...,Major,Resolved,Fixed,{'$oid': '5b48554530a71b06bcb992b0'},"{'sherbold': 'other', 'atrautsch': 'documentat...",other,NaN,NaN,NaN,14723906.0
3,{'$oid': '5bf58b8d423c98412cd404e6'},GIRAPH-7,{'$oid': '5bf58b88423c98412cd404c8'},[],[],[],[],[],Remove Yahoo-specific code from pom.xml,There are remaining references to Y! infrastru...,...,Blocker,Resolved,Duplicate,{'$oid': '5b3b3d6230a71b06bc84d07f'},NaN,NaN,NaN,NaN,NaN,964215916.0
4,{'$oid': '5bf58b8e423c98412cd404ea'},GIRAPH-29,{'$oid': '5bf58b88423c98412cd404c8'},[],[bsp],[],[0.1.0],[],Implement TextVertexInputFormat for text-forma...,Supporting text-format graph data would be nic...,...,Minor,Resolved,Won't Fix,{'$oid': '5b48554530a71b06bcb992b0'},NaN,NaN,NaN,NaN,NaN,18302911.0


""
0
1
2
3
4
5
6
7
8
9
